In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
import json

In [3]:
sc = SparkContext('local[8]', 'task2')

In [4]:
reviews_txt = sc.textFile('asnlib/publicdata/review.json')

In [5]:
reviews_json = reviews_txt.map(lambda x: json.loads(x))

In [6]:
reviews_json.first()

{'review_id': '-I5umRTkhw15RqpKMl_o1Q',
 'user_id': '-mA3-1mN4JIEkqOtdbNXCQ',
 'business_id': 'mRUVMJkUGxrByzMQ2MuOpA',
 'stars': 1.0,
 'text': "Walked in around 4 on a Friday afternoon, we sat at a table just off the bar and walked out after 5 min or so. Don't even think they realized we walked in. However everyone at the bar noticed we walked in!!! Service was non existent at best. Not a good way for a new business to start out. Oh well, the location they are at has been about 5 different things over the past several years, so they will just be added to the list. SMDH!!!",
 'date': '2017-12-15 23:27:08'}

In [7]:
business_review_count = reviews_json.map(lambda x: (x['business_id'], 1)).reduceByKey(lambda a,b: a + b)

In [ ]:
business_review_count.first()

In [8]:
business_review_count.take(20)

[('abaIvBrlg3QI4FUG1v3bdA', 1),
 ('M8a5DRdXl8KMu4bMFLPgQg', 20),
 ('1Df5WnLX3DqN6ymlhqznaQ', 73),
 ('DVfCbJhJUDWRlUfrKzaKOA', 106),
 ('E7eOOXwMC7wwCRywsvNzUA', 27),
 ('VoXUt6TIJWyIf_lihbN5xQ', 51),
 ('6eRSdODyj9GS0w6zlhrWkg', 23),
 ('CN86oDG4uEar1SROyMBKFA', 7),
 ('uf2hyKG-7dyszQZw6n57OQ', 4),
 ('qx6WhZ42eDKmBchZDax4dQ', 173),
 ('WOj8nPos3R9IIf0Pyttd8g', 34),
 ('vA0HlyVJcK8_Wtq81-3TUw', 4),
 ('gc_GGat7TgNCmrgc4nVOgw', 46),
 ('PeVgw94It7HBBLkIbOoAng', 9),
 ('gT4gngtGKYOW-jl5UIBahg', 10),
 ('zra20XPGVL9P3i5hMoKjig', 26),
 ('eoyvbnRYQe-z85e8Rc6vAg', 78),
 ('IqxhMxYSQTg8BytCkaGczA', 3),
 ('DlVALLr6P4_cimALTHjcbg', 9),
 ('vUfFS5BR8WXsx9r72QMCvg', 14)]

In [9]:
reviews_json.getNumPartitions()

27

In [10]:
business_review_count.getNumPartitions()

27

In [11]:
filtered_business_review_count = business_review_count.filter(lambda x: x[1]>50)

In [12]:
results = {}
results['default'] = {}
results['default']['n_parititions'] = reviews_json.getNumPartitions()
results['default']['n_items'] = reviews_json.glom().map(len).collect()
results['default']['result'] = filtered_business_review_count.map(lambda x: list(x)).collect()

In [13]:
results

{'default': {'n_parititions': 27,
  'n_items': [43432,
   43626,
   42795,
   43480,
   42788,
   42803,
   43248,
   42384,
   43461,
   43255,
   42551,
   43224,
   42719,
   42654,
   43355,
   42258,
   43589,
   43759,
   42491,
   43054,
   42732,
   42400,
   42716,
   41857,
   42907,
   43475,
   34612],
  'result': [['1Df5WnLX3DqN6ymlhqznaQ', 73],
   ['DVfCbJhJUDWRlUfrKzaKOA', 106],
   ['VoXUt6TIJWyIf_lihbN5xQ', 51],
   ['qx6WhZ42eDKmBchZDax4dQ', 173],
   ['eoyvbnRYQe-z85e8Rc6vAg', 78],
   ['vyDW7_CHhDUJshMqaSjj0g', 77],
   ['OVTZNSkSfbl3gVB9XQIJfw', 246],
   ['EdY6q0CO2mlATpoTFkvHAw', 71],
   ['biSsdqExmdvIKiFuowOadg', 61],
   ['yAXWKWRTs4wNy-zrJgoeXQ', 52],
   ['7EZ4Eu7YJ1ltRCC5jXFJrQ', 138],
   ['mXyLqsbe7FT1X8ZFOP8WpQ', 77],
   ['qsJkjQoM3Hdg6Gtqp2S9_w', 90],
   ['3l54GTr8-E3XPbIxnF_sAA', 382],
   ['WbJ1LRQdOuYYlRLyTkuuxw', 287],
   ['hH8002bWtOFYtLgarFZBsg', 70],
   ['Sovgwq-E-n6wLqNh3X_rXg', 205],
   ['odyYR1Bg2bqmp1kVlm8OYw', 156],
   ['xlb4QVFWB9CSPT7qNRn5xA', 51],
 

In [ ]:
business_ids = list(reviews_json.map(lambda x: x['business_id']).distinct().collect())

In [14]:
def partition_business_id(id):
    return business_ids.index(id)

In [17]:
# reviews_custom_json = reviews_txt.map(lambda x: json.loads(x)).map(lambda x: (x['business_id'], x)).partitionBy(32, partition_business_id)

In [18]:
# reviews_custom_json.first()

In [33]:
n_partitions = 8

In [34]:
def hash_partition_business_id(id):
    return sum([ord(x) for x in id[:4]]) % n_partitions

In [30]:
def hash_partition_business_id(id):
    return sum([ord(x) for x in id])

In [37]:
reviews_custom_new_json = reviews_json.map(lambda x: (x['business_id'], x)).partitionBy(n_parititions, hash_partition_business_id)

In [38]:
reviews_custom_new_json.first()

('Mem13A3C202RzT53npn4NA',
 {'review_id': 'IPw8yWiyqnfBzzWmypUHgg',
  'user_id': '5JVY32_bmTBfIGpCCsnAfw',
  'business_id': 'Mem13A3C202RzT53npn4NA',
  'stars': 5.0,
  'text': "If you are looking for the best pierogies in Pittsburgh, this is your place. There are a few small tables outside but most of the business is carry out. Pierogies Plus wins Best Pierogies every year. Why? Because the owner is from Poland and she is making the real deal pierogies. The best part is that they are hand pinched by a group of older Polish and Hungarian women. \nThe biggest seller is potato and cheese but they sell many flavors. They are like plump pillows of softness. You can buy them buy the dozen. You can get them cold to take home and freeze or warm and ready to eat. The warm ones are served with butter and onions.  It's definitely a comfort food. The best part is that they ship internationally. Yes, they are that good.",
  'date': '2017-05-13 10:41:43'})

In [41]:
reviews_custom_new_json.map(lambda x: (x[0], 1)).reduceByKey(lambda a,b: a + b).take(40

[('qQCsLGMncJp1xWUGnOKCQA', 100),
 ('rCRy7NfTgnqiey_Z4EU-dA', 6),
 ('99KKeZYRPnz7qDi5USsHmw', 13),
 ('uXzYH9ywBhZ0WwfStrCWNQ', 8),
 ('XAB5nT53CHoGKO--fhLN7g', 3),
 ('zYKblyjRDoiOum6NYuvZ6g', 14),
 ('VUtazCTIc0aoOrQprP_s-Q', 85),
 ('EGzbYCNiLoBuRvJmzqrKxQ', 30),
 ('vezSmTPXG1Tz6DPckHtsCw', 25),
 ('JXmYodz9jw1P23Ygx4OZTQ', 7),
 ('Q4ChVzujWcmoYEKG4-NTeQ', 15),
 ('FKCdzOoetA6A-e3QGraumg', 1),
 ('N8XzoYh6rEdOGL3ZsM0E2A', 1),
 ('OMKa1LTpYQO9XYydgTFRWA', 12),
 ('hhhHgSaSl6GLtuDoDrMUZQ', 24),
 ('2ALi7ZYvil-BAXFa3CMUyA', 33),
 ('HrG_BxmOMPbqstycmzORzw', 3),
 ('aW6Z4NSqEVli-xYMV3jRyg', 16),
 ('BtuMs0o5yCDlSb278sZfNg', 2),
 ('NDC3StqRdWu2Z7WMIBVbBg', 32),
 ('Ql5npMMRWKurN99tgA-g5w', 146),
 ('su5sBvJrgaD5Sh1SnvYzkw', 10),
 ('Hn5UPOrE3VN7r8k4cBS1ug', 3),
 ('baIT89GubjGJV1mpn82Eeg', 14),
 ('KN0gPRzDvA6uVYims2KA0w', 7),
 ('Xk05B3MApfGZB-s3vQkuyA', 1),
 ('l7RCvKqA61Jzm98gaqIhvw', 8),
 ('QAqm1ubKgPYYqjZKxfi87A', 15),
 ('bLZ8_XqjM6AZFYhc7e35HA', 3),
 ('UdC4xRKTFQPOhkc6vPVwvA', 2),
 ('yrC2AxuY3wXNodbuClL